In [1]:
# import packages
import re,os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from IPython.display import clear_output
import pandas as pd
import math
import random
from datetime import datetime
from tqdm import trange
tfk = tf.keras
tfkl = tf.keras.layers
clear_output()
# import my files
from symae_model import SymAE
from plot_save import plot_save
from generate import generate
from plot_training import plot_training
from redatuming import redatuming
from plot_redatuming import plot_redatuming

Num GPUs Available:  2
TensorFlow Version:  2.9.1


In [2]:
# Generate the Dataset
n1 = 3 #ne
n2 = 100
nx = 1000
nt = 20
p = 1
q = 10
dropout_rate = 0
states = np.arange(n1)+1
nuisances = np.arange(n2)+1

In [3]:
data = generate(states,nuisances,nx,nt,outer_replace=True)
X = data.X
X_states = data.X_states
X_nuisances = data.X_nuisances

In [4]:
# Initialize SymAE
model = SymAE(nx,nt,p,q,dropout_rate)
model.load_weights('./checkpoints/'+datetime.now().strftime("%B%d"))
#model.load_weights('./checkpoints/'+'June16')
clear_output()

2022-06-20 12:58:49.257925: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-20 12:58:50.381911: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14254 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:b1:00.0, compute capability: 8.6
2022-06-20 12:58:50.382752: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14254 MB memory:  -> device: 1, name: NVIDIA RTX A4000, pci bus id: 0000:b2:00.0, compute capability: 8.6


ValueError: Received incompatible tensor with shape (4,) when attempting to restore variable with shape (11,) and name decoder/layer_with_weights-0/layer_with_weights-0/bn/beta/.ATTRIBUTES/VARIABLE_VALUE.

In [ ]:
Adam = tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-07)
model.compile(loss='mse',optimizer=Adam)

In [ ]:
# Train the NeuralNet
M=1000
epochs=range(M)
losses=[0.0]*M
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        losses[epoch]=logs["loss"]
        print("For epoch {:d}, loss is {:f}.".format(epoch, logs["loss"]))
history=model.fit(X,X,epochs=M,verbose=0,callbacks=[CustomCallback()])

In [ ]:
plt.plot(epochs,losses)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
# Save weights
model.save_weights('./checkpoints/'+datetime.now().strftime("%B%d"))
print("weights saved")

In [ ]:
# Plot the result
Y = model.predict(X)
i = 0
fig = plot_training(X,Y,i)
plt.title('i=%d, state=%d'%(0,X_states[0]))

In [ ]:
# Save plot
plot_save(fig,ID=-1)

In [ ]:
X_states[0:5]

In [ ]:
# Redatuming
Z, Z_hat = redatuming(model,data)

In [ ]:
i = 0
i_prime = 1
fig = plot_redatuming(Z,Z_hat,X_states,i,i_prime)

In [ ]:
# Save plot
plot_save(fig,ID=-1)

In [ ]:
def CEnc(Array_1D_len_nt,j=-1):
    return model.sym_encoder.predict(Array_1D_len_nt[np.newaxis, :, np.newaxis],verbose=0)[0,:]
def NEnc(Array_1D_len_nt,j):
    return model.nui_encoder.predict(Array_1D_len_nt[np.newaxis, :, np.newaxis],verbose=0)[0,j*q : (j+1)*q]